In [2]:
import pandas as pd
import ast
import gzip

In [3]:
lista = []

# Abrir el archivo y procesar cada línea
with gzip.open(r'C:\Users\argui\OneDrive\Escritorio\ProyectoML_OPS\data\users_items.json.gz', 'rt', encoding='utf-8') as f:
    for linea in f:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(linea)
            lista.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {linea}")
            continue

#Creamos un DataFrame a partir de la lista de diccionarios
df_items = pd.DataFrame(lista)     
df_items.head(3)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


Eliminamos algunas columnas que no necesitaremos

In [4]:
df_items = df_items.drop(columns='user_url')
df_items = df_items.drop(columns='items_count')
df_items = df_items.drop(columns='steam_id')
df_items.head(3)

,user_id,items
0,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


Desanidamos la columna items

In [5]:
#vamos a expandir el df creando una fila por cada elemento anidado en items
df_items = df_items.explode('items').reset_index()

#ahora creamos una nueva columna por cada elemento de la lista en la columna items
df_items = pd.concat([df_items,pd.json_normalize(df_items['items'])],axis=1)

#eliminamos las columbas index y la columna items
df_items.drop(columns=['items'],inplace=True)
df_items.drop(columns=['index'],inplace=True)

df_items.head(5)

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6.0,0.0
1,76561197970982479,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,30,Day of Defeat,7.0,0.0
3,76561197970982479,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0


Comenzamos con la limpieza de los datos

In [6]:
#Verificamos la cantidad de nulos en item_id
df_items['item_id'].isna().value_counts()

item_id
False    5153209
True       16806
Name: count, dtype: int64

In [7]:
df_items = df_items.dropna(subset='item_id')
df_items.isna().sum()

user_id             0
item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
dtype: int64

In [8]:
df_items = df_items.drop(columns='user_id')

Eliminamos duplicados

In [9]:
df_items_1 = df_items.drop_duplicates(keep='first')
print(df_items.shape)
print(df_items_1.shape)

(5153209, 4)
(1442052, 4)


Finalmente convertimos a parquet

In [10]:
df_items_1.to_parquet(r'C:\Users\argui\OneDrive\Escritorio\ProyectoML_OPS\data\users_items.parquet',compression='snappy',index=False)